In [3]:

import os
import sys

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "Colab Notebooks/prj_neuroread_analysis/neuroread/"
GOOGLE_DRIVE_PATH = os.path.join("/content", "drive", "MyDrive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

# Add to sys so we can import .py files.
sys.path.append(GOOGLE_DRIVE_PATH)
os.chdir(GOOGLE_DRIVE_PATH)

# Install unavailable packages
import pip
def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])

import_or_install("mne")


['.git', '.DS_Store', '.gitignore', 'EEG', 'LICENSE', 'train_cl_eeg2speech_rochester_v1.ipynb', 'train_cl_eeg2speech_rochester_v2.ipynb', '.ipynb_checkpoints', 'train_cl_eeg2speech_rochester_v3_test_old.ipynb', 'runs', 'train_cl_eeg2speech_rochester_v3_test.ipynb', 'train_cl_eeg2speech_rochester_v4_gridseaerch.ipynb', 'train_cl_eeg2speech_2.ipynb', 'train_cl_eeg2speech_rochester_subj_2.ipynb', 'README.md', 'train_eeg2speech_rochester.ipynb', 'train_cl_eeg2speech_rochester_v3.ipynb', 'models', 'run_training_gridsearch.py', 'tools', 'train_cl_eeg2speech_rochester_gridsearch.ipynb', 'train_cl_eeg2speech_rochester_gridsearch_test.ipynb', 'train_cl_eeg2speech_rochester_v4_gridsearch.ipynb', 'run_training_gridserch.ipynb']


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

Collecting mne

Downloading mne-1.3.1-py3-none-any.whl (7.6 MB)

...

Requirement already satisfied: urllib3<1.27,>=1.21.1 in /usr/local/lib/python3.8/dist-packages (from requests>=2.19.0->pooch>=1.5->mne) (1.26.14)

Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.8/dist-packages (from requests>=2.19.0->pooch>=1.5->mne) (2.10)

Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.8/dist-packages (from requests>=2.19.0->pooch>=1.5->mne) (2022.12.7)

Installing collected packages: mne

Successfully installed mne-1.3.1

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
 print('Not connected to a GPU')
else:
 print(gpu_info)

Your runtime has 89.6 gigabytes of available RAM

Sat Mar  4 07:53:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                      

In [7]:
import os, sys, glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np

import mne
import time


from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

from tools.train import eval_model_cl
from tools.data import unfold_raw, rm_repeated_annotations
from tools.load_data import load_data

from models.eeg_encoder import EEGEncoder
from models.envelope_encoder import EnvelopeEncoder
from models.contrastive_eeg_speech import CLEE
# -------------------------------------

cuda:0


In [8]:






# Read the command-line argument passed to the interpreter when invoking the script
lr = 0.001#sys.argv[1]
batch_size = 64# sys.argv[2]


# Read data
subj_ids = list(range(1, 20))
fs = 128
window_size = int(5 * fs)
stride_size_train, stride_size_val, stride_size_test = int(2.5 * fs), int(5 * fs), int(5 * fs)
batch_size = int(batch_size)
lr = lr

n_channs = 129 # 128 for eeg, 1 for env
print('-------------------------------------')
print(f'window_size: {window_size}  stride_size_test: {stride_size_test}')

dataset_name = ['rochester_data', 'natural_speech']
outputs_path = f'../outputs/'
data_path = os.path.join(outputs_path, dataset_name[0], dataset_name[1])
after_ica_path = os.path.join(data_path, 'after_ica_raw')
print(f'data_path: {data_path}')


X = load_data(subj_ids, after_ica_path, window_size, 
              stride_size_train, stride_size_val, stride_size_test, n_channs)


# Create dataloaders
class MyDataset(Dataset):
    def __init__(self, eeg, env):
        self.eeg = eeg
        self.env = env
    
    def __getitem__(self, index):
        return self.eeg[index], self.env[index]
    
    def __len__(self):
        return len(self.eeg)
    

dataset_train = MyDataset(X['eegs_train'], X['envs_train'])
dl_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
dl_val = DataLoader(MyDataset(X["eegs_val"], X["envs_val"]), 
                    batch_size=batch_size, shuffle=True, drop_last=True)



# Create model
eeg_encoder = EEGEncoder()
env_encoder = EnvelopeEncoder()
model = CLEE(eeg_encoder, env_encoder)
model.to(device)

# Train model
models_dict = {f'lr_{lr}_bs_{batch_size}': model}
lossi = []
udri = [] # update / data ratio 
ud = []



for name, model in models_dict.items():

    # Reset for the new model in the loop
    print(f"+--------------New model: {name}----------------------+")
    writer = SummaryWriter(log_dir=f"runs/{name}_{time.strftime('%Y%m%d_%H%M%S')}")
    model.to(device)
    optimizer = optim.NAdam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.7)
    cnt = 0
    loss_batches = []


    for epoch in range(1, 400):

        print(f"====== Epoch: {epoch}")

        model.train()
        for ix_batch, (Xb_eeg, Xb_env) in enumerate(dl_train):

            # send to device
            Xb_eeg = Xb_eeg.to(device)
            Xb_env = Xb_env.to(device)

            # Zero out gradients
            optimizer.zero_grad()

            # forward pass
            eeg_features, env_features, logit_scale = model(Xb_eeg, Xb_env) 


            # normalize features
            eeg_features_n = eeg_features / eeg_features.norm(dim=1, keepdim=True)
            env_features_n = env_features / env_features.norm(dim=1, keepdim=True)

            # logits
            logits_per_eeg = logit_scale * eeg_features_n @ env_features_n.t()
            logits_per_env = logits_per_eeg.t()

            #loss function
            labels = torch.arange(batch_size).to(device)
            loss_eeg = F.cross_entropy(logits_per_eeg, labels)
            loss_env = F.cross_entropy(logits_per_env, labels)
            loss   = (loss_eeg + loss_env)/2

            # backward pass
            loss.backward()
            optimizer.step()

            loss_batches.append(loss.item())
            cnt += 1

            with torch.no_grad():
                #ud = {f"p{ix}":(lr*p.grad.std() / p.data.std()).log10().item() for ix, p in enumerate(model.parameters()) if p.ndim==4 }
                #writer.add_scalars('UpdateOData/ud', ud, cnt)
                writer.add_scalar('Loss/train_batch', loss.item(), cnt)

            # normalize weights
            with torch.no_grad():
                model.eeg_encoder.normalize_weights()
            
            #break   

        loss_epoch = loss_batches[-(ix_batch + 1):]  # mean loss across batches
        loss_epoch = sum(loss_epoch) / len(loss_epoch)
        writer.add_scalar('Loss/train_epoch', loss_epoch, epoch)
        #for pname, p in model.named_parameters():
        #writer.add_histogram(f'Params/{pname}', p, epoch)
        #writer.add_histogram(f'Grads/{pname}', p.grad, epoch)

        loss_val, *_ = eval_model_cl(dl_val, model, device=device)
        writer.add_scalar('Loss/val_epoch', loss_val, epoch)

        

        model.train()

        # Update learning rate based on epoch
        scheduler.step()
            
    #break   







-------------------------------------
window_size: 640  stride_size_test: 640
data_path: ../outputs/rochester_data/natural_speech
Opening raw data file ../outputs/rochester_data/natural_speech/after_ica_raw/subj_1_after_ica_raw.fif...
    Range : 0 ... 464571 =      0.000 ...  3629.461 secs
Ready.
Reading 0 ... 464571  =      0.000 ...  3629.461 secs...
Initial num of annots: 48  Num of removed annots: 19  Num of retained annots:  29
-------------------------------------
N train: 26  N val: 1  N test: 1
Opening raw data file ../outputs/rochester_data/natural_speech/after_ica_raw/subj_2_after_ica_raw.fif...
    Range : 0 ... 464571 =      0.000 ...  3629.461 secs
Ready.
Reading 0 ... 464571  =      0.000 ...  3629.461 secs...
Initial num of annots: 65  Num of removed annots: 19  Num of retained annots:  46
-------------------------------------
N train: 68  N val: 2  N test: 2
Opening raw data file ../outputs/rochester_data/natural_speech/after_ica_raw/subj_3_after_ica_raw.fif...
    Ran

Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.

NumExpr defaulting to 8 threads.

====== Epoch: 1


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv2d(input, weight, bias, self.stride,


====> Validation loss: 4.0448,  X1 loss: 4.0306   X2 loss: 4.0591
====== Epoch: 2
====> Validation loss: 3.9165,  X1 loss: 3.9073   X2 loss: 3.9258
====== Epoch: 3
====> Validation loss: 3.8540,  X1 loss: 3.8411   X2 loss: 3.8669
====== Epoch: 4
====> Validation loss: 3.8241,  X1 loss: 3.8112   X2 loss: 3.8370
====== Epoch: 5
====> Validation loss: 3.7721,  X1 loss: 3.7609   X2 loss: 3.7833
====== Epoch: 6
====> Validation loss: 3.7368,  X1 loss: 3.7317   X2 loss: 3.7419
====== Epoch: 7
====> Validation loss: 3.7208,  X1 loss: 3.7148   X2 loss: 3.7268
====== Epoch: 8
====> Validation loss: 3.7135,  X1 loss: 3.7018   X2 loss: 3.7252
====== Epoch: 9
====> Validation loss: 3.7112,  X1 loss: 3.6989   X2 loss: 3.7235
====== Epoch: 10
====> Validation loss: 3.6966,  X1 loss: 3.6860   X2 loss: 3.7071
====== Epoch: 11
====> Validation loss: 3.6853,  X1 loss: 3.6771   X2 loss: 3.6934
====== Epoch: 12
====> Validation loss: 3.7226,  X1 loss: 3.7076   X2 loss: 3.7376
====== Epoch: 13
====> Valida